In [ ]:
import numpy as np
from sklearn import preprocessing


In [ ]:
raw_csv_data = open('/content/drive/MyDrive/audiobook.csv')

In [ ]:
raw_csv_data = np.loadtxt('/content/drive/MyDrive/audiobook.csv',delimiter=',')

In [ ]:
unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [ ]:
# Balancing the dataset here we have targets as 1 and 0  1 are less , so we'll count 1s eg 15 and we'll keep as many 0s as 1s i.e 0s also 15

In [ ]:
num_one_targets = int(np.sum(targets_all))
zeros_targets_counter = 0
indices_to_remove=[]


for i in range(targets_all.shape[0]):
  if targets_all[i] ==0:
    zeros_targets_counter+=1
    if zeros_targets_counter > num_one_targets:
      indices_to_remove.append(i)


unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)


**Standardize the inputs**


In [ ]:
df

**Shuffle the Data**

In [ ]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

#Split the Dataset into Train, Validation and Test

.

In [ ]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 *samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count


train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets),train_samples_count,np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets),validation_samples_count,np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets),test_samples_count,np.sum(test_targets)/test_samples_count)

1790.0 3579 0.5001397038278849
226.0 447 0.5055928411633109
221.0 448 0.49330357142857145


#Save the three datasets in *.npz

In [ ]:
np.savez('audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('audiobooks_data_test',inputs=test_inputs,targets=test_targets)

#Creating the Machine Learning Algorithm

In [ ]:
import tensorflow as tf


#Load the Data

In [ ]:
npz = np.load('audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets=npz['targets'].astype(np.int)

npz=np.load('audiobooks_data_validation.npz')
validation_inputs,validation_targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

npz=np.load('audiobooks_data_test.npz')
test_inputs,test_targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)


#Model  outline,optimizers,loss,early stopping and training

In [ ]:
input_size=10
output_size=2
hidden_layer_size=50
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size=100
max_epochs=100
early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2)

Epoch 1/100
36/36 - 1s - loss: 0.6013 - accuracy: 0.6600 - val_loss: 0.4953 - val_accuracy: 0.7785 - 605ms/epoch - 17ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4715 - accuracy: 0.7689 - val_loss: 0.4189 - val_accuracy: 0.7875 - 95ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4136 - accuracy: 0.7840 - val_loss: 0.3827 - val_accuracy: 0.8233 - 94ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3843 - accuracy: 0.8025 - val_loss: 0.3675 - val_accuracy: 0.8031 - 106ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3698 - accuracy: 0.8027 - val_loss: 0.3586 - val_accuracy: 0.8255 - 82ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3620 - accuracy: 0.8039 - val_loss: 0.3481 - val_accuracy: 0.8233 - 107ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3539 - accuracy: 0.8100 - val_loss: 0.3472 - val_accuracy: 0.8121 - 110ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3484 - accuracy: 0.8103 - val_loss: 0.3420 - val_accuracy: 0.8255 - 88ms/epoch - 2ms/step
Epoch 9/100

#Test Model

In [ ]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.8438
